# A User Perception sub-system.

The aim of this sub-system is to acquire input from a webcam in real time, of a human user or a group of users interacting with Furhat, and to automatically detect their affective states. The basic requirements for this sub-system are to: (i) automatically extract facial features (e.g. Action Units) from the video input using the tools you have learned in the course (e.g., Py-Feat), and (ii) use Machine Learning (ML) techniques to process the features into a high-level representation of the user’s behaviour (i.e., affective states: valence or arousal, or both) that can be used by the second sub-system. You are free to choose which facial features to extract, which ML techniques to apply, which affective state to model, and whether to automatically detect the affective state of one user or a group of users. In order to train the ML model for the automatic detection of affective states, you may use any dataset that you find but we will only provide the MultiEmoVA dataset (if you choose to automatically detect the affective state of a group of users in your scenario) or the DiffusionFER dataset (if you choose to automatically detect the affective state of one single user in your scenario). Regardless of your choice of dataset, you will need to extract your own features. Images and labels for these datasets will be made available in Studium.

In [ ]:
import cv2
import numpy as np
from feat import Detector
import opencv_jupyter_ui as jcv2
import torch

In [ ]:
import logging, sys
logging.disable(sys.maxsize)

In [ ]:
detector = Detector(device='cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def feeling(image, enable_aus=False, enable_emotion=False):
    face = detector.detect_faces(image)

    aus = None
    emotion = None

    if enable_emotion or enable_aus:
        landmark = detector.detect_landmarks(image, face)
        if enable_aus:
            aus = detector.detect_aus(image, landmark)
        if enable_emotion:
            emotion_int = detector.detect_emotions(image, face, landmark)

    for (i, face) in enumerate(face[0]):
        f_hl = int(face[0])       # horizontal left
        f_vt = int(face[1])       # vertical top
        f_hr = int(face[2])       # horizontal right
        f_vb = int(face[3])       # vertical bottom

        color = (0, 255, 0)
        thickness = 2
        cv2.line(image,(f_hl, f_vt), (f_hr, f_vt), color, thickness)   # top left to top right
        cv2.line(image,(f_hr, f_vt), (f_hr, f_vb), color, thickness)   # top right to bottom right
        cv2.line(image,(f_hr, f_vb), (f_hl, f_vb), color, thickness)   # bottom right to bottom left
        cv2.line(image,(f_hl, f_vb), (f_hl, f_vt), color, thickness)   # bottom left to top right

        if enable_emotion:
            emotion = "NONE"
            match np.argmax(emotion_int[0][i]):
                case 0:
                    emotion = "anger"
                case 1:
                    emotion = "disgust"
                case 2:
                    emotion = "fear"
                case 3:
                    emotion = "happiness"
                case 4:
                    emotion = "sadness"
                case 5:
                    emotion = "surprise"
                case 6:
                    emotion = "neutral"

            cv2.putText(image, emotion, (f_hl, f_vt-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color)

    return image, aus, emotion

In [ ]:
cam = cv2.VideoCapture(0)
counter = 0

while True:
    # check = True means we managed to get a frame.
    # If check = False, the device is not available, and we should quit.
    check, frame = cam.read()
    if not check:
        break

    new_frame, aus, emotion = feeling(frame, False, False)

    # OpenCV uses a separate window to display output.
    jcv2.imshow("video", new_frame)

    # Press ESC to exit.
    key = jcv2.waitKey(1) & 0xFF
    if key == 27:
        break

cam.release()
jcv2.destroyAllWindows()